In [1]:
# https://stackoverflow.com/questions/21006940/how-to-load-all-entries-in-an-infinite-scroll-at-once-to-parse-the-html-in-pytho?noredirect=1&lq=1
# https://github.com/SeleniumHQ/docker-selenium/issues/87

import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

browser = webdriver.Chrome("chromedriver.exe")
browser.get("https://www.kaggle.com/rankings?group=competitions")
time.sleep(1)

elem = browser.find_element_by_tag_name("body")

no_of_pagedowns = 200  # *10 ~= total #experimentar 190 para dar 3000 certos

while no_of_pagedowns:
    elem.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.2)
    no_of_pagedowns-=1

post_elems = browser.find_elements_by_class_name("block-link__anchor")

urls = []
for post in post_elems:
    urls.append(post.get_attribute("href"))
    
print(len(urls))

2960


In [3]:
# http://www.pythonforbeginners.com/python-on-the-web/web-scraping-with-beautifulsoup/
from bs4 import BeautifulSoup
import requests, sys

cities = []
countries = []
print("Collecting data...")
for i, url in enumerate(urls):
    sys.stdout.write("\r%.2f%%" % (100*(i+1)/len(urls)))
    r = requests.get(url)
    data = r.text
    start_country = data.find("country")
    end_country = data[start_country:].find(",")
    country = data[start_country + 10: start_country + end_country - 1]
    if country == 'ul':
        country = ''
    countries.append(country)
    start_city = data.find("city")
    end_city = data[start_city:].find(",")
    city = data[start_city + 7: start_city + end_city - 1]
    if city == 'ul':
        city = ''
    cities.append(city)
    time.sleep(0.2)
    sys.stdout.flush()
print("Done.")

1.00%Done.


In [4]:
import pandas as pd

df = pd.DataFrame({'Link': urls, 'Country': countries, 'City': cities})
df[df.City=='ul']["City"] = ''
df[df.Country=='ul']["Country"] = ''
df.head()

,City,Country,Link
0,São Francisco,Estados Unidos,https://www.kaggle.com/titericz
1,Volos,GR,https://www.kaggle.com/kazanova
2,Moscow,Russian Federation,https://www.kaggle.com/stasg7
3,,,https://www.kaggle.com/bestfitting
4,Deutschland,,https://www.kaggle.com/mmueller


In [5]:
df.to_csv('data.csv', encoding='utf-8', index=False)
print('Saved')

Saved
